In [55]:
import pandas as pd
import re
import pymongo

In [56]:
# Kết nối tới MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")  
db = client["tgdd_db"]  
collection = db["dhdt_product"]  

In [57]:
# Trích xuất dữ liệu từ MongoDB và bỏ qua trường '_id'
cursor = collection.find({}, {'_id': 0})

# Chuyển dữ liệu từ MongoDB sang DataFrame của Pandas
df = pd.DataFrame(list(cursor))

# Hiển thị 5 dòng đầu tiên
df.head()

,Product ID,Name,Price,Old Price,Thời gian sử dụng pin:,Đối tượng sử dụng:,Đường kính mặt:,Độ dày mặt:,Kháng nước:,Nguồn năng lượng:,Chất liệu mặt kính:,Thương hiệu của:,Hãng:,Quantity Sale,Discount,Rating,Review Count,Bình luận
0,242206,MVW 40 mm Nam MS060-01,990000.0,1990000.0,Khoảng 2 năm,Nam,40 mm,9 mm,"5 ATM - Đi mưa, tắm",Pin,Kính Sapphire,Việt Nam,MVW.,184,-50%,4.6,20 đánh giá,"Đồng hồ đúng giao đúng mẫu, đúng màu ,đẹp ,từ ..."
1,269354,ELIO 33 mm Nữ EC006-02,690000.0,1090000.0,Khoảng 2 năm,Nữ,33 mm,7.7 mm,"3 ATM - Rửa tay, đi mưa",Pin,Kính khoáng Mineral,Việt Nam,ELIO.,32,-36%,4.1,15 đánh giá,Rất ưng ý
2,324772,SMILE KID 38 mm Trẻ em SL414-012SK,290000.0,390000.0,Khoảng 1.5 năm,Trẻ em,38 mm,14 mm,"5 ATM - Đi mưa, tắm",Pin,Nhựa Resin,Việt Nam,SMILE KID.,"1,1k",-25%,None,0,N/A
3,208661,CASIO 42 mm Nam MTP-1375L-7AVDF,1702000.0,2270000.0,Khoảng 2 năm,Nam,42 mm,Hãng không công bố,"5 ATM - Đi mưa, tắm",Pin,Kính khoáng Mineral,Nhật Bản,CASIO.,219,-25%,4.8,14 đánh giá,Mẫu mã đẹp nhìn chất lượng rất tốt trong tầm giá
4,228864,CASIO 43.5 mm Nam AE-1000W-1BVDF,852000.0,1136000.0,Khoảng 10 năm,Nam,43.5 mm,13.5 mm,"10 ATM - Tắm, bơi",Pin,Nhựa Resin,Nhật Bản,CASIO.,221,-25%,4.3,11 đánh giá,máy ok. cái dây nó có khớp đeo thật là đau tay


In [58]:
import re

def process_name(name):
    # Loại bỏ từ "Mẫu mới" nếu có
    name = name.replace("Mẫu mới", "").strip()
    
    # Loại bỏ các chuỗi kích thước như 'xx mm', 'xx x yy mm', 'xx.x mm', hoặc 'xx × yy mm'
    # và các thông tin giới tính như 'Nam', 'Nữ', 'Trẻ em', 'Unisex'
    clean_name = re.sub(r'\d+(\.\d+)?( x \d+(\.\d+)?| × \d+(\.\d+)?)*\s*mm|Nam|Nữ|Trẻ em|Unisex', '', name).strip()

    # Loại bỏ dấu gạch nối hoặc khoảng trắng không cần thiết
    clean_name = re.sub(r'\s*-\s*', '', clean_name).strip()

    # Loại bỏ từ "Eco-Drive" nếu có ở cuối chuỗi
    clean_name = re.sub(r'Eco-Drive$', '', clean_name).strip()

    # Tìm mã sản phẩm dựa trên định dạng mã (các chữ và số cuối)
    product_code = re.search(r'[A-Z0-9\/\-\.]+$', clean_name)
    
    # Nếu tìm thấy mã sản phẩm, kết hợp lại với tên sản phẩm
    if product_code:
        name_only = clean_name.replace(product_code.group(), '').strip()
        code = product_code.group()
        # Kết hợp tên sản phẩm và mã sản phẩm vào một chuỗi
        result = f"{name_only} {code}"
    else:
        # Nếu không tìm thấy mã sản phẩm, chỉ giữ tên sản phẩm
        result = clean_name
    
    # Trả về tên sản phẩm và mã sản phẩm gộp lại
    return result

# Áp dụng hàm xử lý cho cột 'Name' và cập nhật lại cột này
df['Tên sản phẩm'] = df['Name'].apply(process_name)


In [59]:
# Xóa cột 'Name' khỏi dataframe
df.drop(columns=['Name'], inplace=True)

In [60]:
df.head(5)

,Product ID,Price,Old Price,Thời gian sử dụng pin:,Đối tượng sử dụng:,Đường kính mặt:,Độ dày mặt:,Kháng nước:,Nguồn năng lượng:,Chất liệu mặt kính:,Thương hiệu của:,Hãng:,Quantity Sale,Discount,Rating,Review Count,Bình luận,Tên sản phẩm
0,242206,990000.0,1990000.0,Khoảng 2 năm,Nam,40 mm,9 mm,"5 ATM - Đi mưa, tắm",Pin,Kính Sapphire,Việt Nam,MVW.,184,-50%,4.6,20 đánh giá,"Đồng hồ đúng giao đúng mẫu, đúng màu ,đẹp ,từ ...",MVW MS06001
1,269354,690000.0,1090000.0,Khoảng 2 năm,Nữ,33 mm,7.7 mm,"3 ATM - Rửa tay, đi mưa",Pin,Kính khoáng Mineral,Việt Nam,ELIO.,32,-36%,4.1,15 đánh giá,Rất ưng ý,ELIO EC00602
2,324772,290000.0,390000.0,Khoảng 1.5 năm,Trẻ em,38 mm,14 mm,"5 ATM - Đi mưa, tắm",Pin,Nhựa Resin,Việt Nam,SMILE KID.,"1,1k",-25%,None,0,N/A,SMILE KID SL414012SK
3,208661,1702000.0,2270000.0,Khoảng 2 năm,Nam,42 mm,Hãng không công bố,"5 ATM - Đi mưa, tắm",Pin,Kính khoáng Mineral,Nhật Bản,CASIO.,219,-25%,4.8,14 đánh giá,Mẫu mã đẹp nhìn chất lượng rất tốt trong tầm giá,CASIO MTP1375L7AVDF
4,228864,852000.0,1136000.0,Khoảng 10 năm,Nam,43.5 mm,13.5 mm,"10 ATM - Tắm, bơi",Pin,Nhựa Resin,Nhật Bản,CASIO.,221,-25%,4.3,11 đánh giá,máy ok. cái dây nó có khớp đeo thật là đau tay,CASIO AE1000W1BVDF


In [61]:
import re

def process_discount(discount):
    # Kiểm tra xem giá trị có phải là phần trăm với dấu trừ hay không
    if isinstance(discount, str):
        # Xử lý các giá trị kiểu '-50%' hoặc '50%'
        match = re.match(r'(-?\d+)%', discount)
        if match:
            # Chuyển phần trăm thành dạng thập phân và loại bỏ dấu %
            return abs(int(match.group(1))) / 100  # Chuyển thành thập phân
    elif isinstance(discount, (int, float)):
        # Nếu giá trị là số thập phân, không cần thay đổi gì
        return abs(discount)  # Nếu là số thập phân, chỉ lấy giá trị tuyệt đối

    return None  # Nếu không phải dạng phần trăm hoặc số, trả về None

# Áp dụng hàm xử lý cho cột 'Discount'
df['Discount'] = df['Discount'].apply(process_discount)


In [62]:
df.head(3)

,Product ID,Price,Old Price,Thời gian sử dụng pin:,Đối tượng sử dụng:,Đường kính mặt:,Độ dày mặt:,Kháng nước:,Nguồn năng lượng:,Chất liệu mặt kính:,Thương hiệu của:,Hãng:,Quantity Sale,Discount,Rating,Review Count,Bình luận,Tên sản phẩm
0,242206,990000.0,1990000.0,Khoảng 2 năm,Nam,40 mm,9 mm,"5 ATM - Đi mưa, tắm",Pin,Kính Sapphire,Việt Nam,MVW.,184,0.50,4.6,20 đánh giá,"Đồng hồ đúng giao đúng mẫu, đúng màu ,đẹp ,từ ...",MVW MS06001
1,269354,690000.0,1090000.0,Khoảng 2 năm,Nữ,33 mm,7.7 mm,"3 ATM - Rửa tay, đi mưa",Pin,Kính khoáng Mineral,Việt Nam,ELIO.,32,0.36,4.1,15 đánh giá,Rất ưng ý,ELIO EC00602
2,324772,290000.0,390000.0,Khoảng 1.5 năm,Trẻ em,38 mm,14 mm,"5 ATM - Đi mưa, tắm",Pin,Nhựa Resin,Việt Nam,SMILE KID.,"1,1k",0.25,None,0,N/A,SMILE KID SL414012SK


In [63]:
import pandas as pd
import re

def process_battery_life(battery_life):
    if pd.isna(battery_life) or battery_life == '':
        return pd.NA  # Trả về NaN cho giá trị rỗng hoặc None
    
    months = 0  # Khởi tạo biến lưu số tháng
    if 'tháng' in battery_life:
        months_match = re.search(r'(\d+(\.\d+)?)', battery_life)  # Tìm số tháng
        if months_match:
            months += float(months_match.group(1))  # Cộng thêm số tháng (có thể là số thập phân)
    elif 'năm' in battery_life:
        years_match = re.search(r'(\d+(\.\d+)?)', battery_life)  # Tìm số năm
        if years_match:
            months += float(years_match.group(1)) * 12  # Chuyển đổi năm thành tháng
    
    return months  # Trả về số tháng dưới dạng float

# Xử lý cột 'Thời gian sử dụng pin'
df['Thời gian sử dụng pin:'] = df['Thời gian sử dụng pin:'].apply(process_battery_life)


In [64]:
# Đổi tên cột thành "Thời gian sử dụng pin (năm)"
df.rename(columns={'Thời gian sử dụng pin:': 'Thời gian sử dụng pin (tháng)'}, inplace=True)

In [65]:
df.head()

,Product ID,Price,Old Price,Thời gian sử dụng pin (tháng),Đối tượng sử dụng:,Đường kính mặt:,Độ dày mặt:,Kháng nước:,Nguồn năng lượng:,Chất liệu mặt kính:,Thương hiệu của:,Hãng:,Quantity Sale,Discount,Rating,Review Count,Bình luận,Tên sản phẩm
0,242206,990000.0,1990000.0,24.0,Nam,40 mm,9 mm,"5 ATM - Đi mưa, tắm",Pin,Kính Sapphire,Việt Nam,MVW.,184,0.50,4.6,20 đánh giá,"Đồng hồ đúng giao đúng mẫu, đúng màu ,đẹp ,từ ...",MVW MS06001
1,269354,690000.0,1090000.0,24.0,Nữ,33 mm,7.7 mm,"3 ATM - Rửa tay, đi mưa",Pin,Kính khoáng Mineral,Việt Nam,ELIO.,32,0.36,4.1,15 đánh giá,Rất ưng ý,ELIO EC00602
2,324772,290000.0,390000.0,18.0,Trẻ em,38 mm,14 mm,"5 ATM - Đi mưa, tắm",Pin,Nhựa Resin,Việt Nam,SMILE KID.,"1,1k",0.25,None,0,N/A,SMILE KID SL414012SK
3,208661,1702000.0,2270000.0,24.0,Nam,42 mm,Hãng không công bố,"5 ATM - Đi mưa, tắm",Pin,Kính khoáng Mineral,Nhật Bản,CASIO.,219,0.25,4.8,14 đánh giá,Mẫu mã đẹp nhìn chất lượng rất tốt trong tầm giá,CASIO MTP1375L7AVDF
4,228864,852000.0,1136000.0,120.0,Nam,43.5 mm,13.5 mm,"10 ATM - Tắm, bơi",Pin,Nhựa Resin,Nhật Bản,CASIO.,221,0.25,4.3,11 đánh giá,máy ok. cái dây nó có khớp đeo thật là đau tay,CASIO AE1000W1BVDF


In [66]:
df.head()

,Product ID,Price,Old Price,Thời gian sử dụng pin (tháng),Đối tượng sử dụng:,Đường kính mặt:,Độ dày mặt:,Kháng nước:,Nguồn năng lượng:,Chất liệu mặt kính:,Thương hiệu của:,Hãng:,Quantity Sale,Discount,Rating,Review Count,Bình luận,Tên sản phẩm
0,242206,990000.0,1990000.0,24.0,Nam,40 mm,9 mm,"5 ATM - Đi mưa, tắm",Pin,Kính Sapphire,Việt Nam,MVW.,184,0.50,4.6,20 đánh giá,"Đồng hồ đúng giao đúng mẫu, đúng màu ,đẹp ,từ ...",MVW MS06001
1,269354,690000.0,1090000.0,24.0,Nữ,33 mm,7.7 mm,"3 ATM - Rửa tay, đi mưa",Pin,Kính khoáng Mineral,Việt Nam,ELIO.,32,0.36,4.1,15 đánh giá,Rất ưng ý,ELIO EC00602
2,324772,290000.0,390000.0,18.0,Trẻ em,38 mm,14 mm,"5 ATM - Đi mưa, tắm",Pin,Nhựa Resin,Việt Nam,SMILE KID.,"1,1k",0.25,None,0,N/A,SMILE KID SL414012SK
3,208661,1702000.0,2270000.0,24.0,Nam,42 mm,Hãng không công bố,"5 ATM - Đi mưa, tắm",Pin,Kính khoáng Mineral,Nhật Bản,CASIO.,219,0.25,4.8,14 đánh giá,Mẫu mã đẹp nhìn chất lượng rất tốt trong tầm giá,CASIO MTP1375L7AVDF
4,228864,852000.0,1136000.0,120.0,Nam,43.5 mm,13.5 mm,"10 ATM - Tắm, bơi",Pin,Nhựa Resin,Nhật Bản,CASIO.,221,0.25,4.3,11 đánh giá,máy ok. cái dây nó có khớp đeo thật là đau tay,CASIO AE1000W1BVDF


In [67]:
import pandas as pd
import re

def process_water_resistance(resistance):
    if pd.isna(resistance):  # Kiểm tra giá trị NaN
        return None
    
    # Kiểm tra xem có giá trị theo định dạng 'xx ATM' hay không
    match = re.match(r'(\d+)\s*ATM', resistance)  
    if match:
        level = int(match.group(1))  # Chuyển đổi mức kháng nước thành số
        return level
    
    return None  # Trả về None nếu không khớp với định dạng mong muốn

# Áp dụng hàm xử lý cho cột 'Kháng nước:'
df['Mức kháng nước'] = df['Kháng nước:'].apply(process_water_resistance)

# Đổi tên cột 'Mức kháng nước' để thêm đơn vị 'ATM'
df.rename(columns={'Mức kháng nước': 'Mức kháng nước (ATM)'}, inplace=True)

# Xóa cột 'Kháng nước:' khỏi DataFrame
df.drop(columns=['Kháng nước:'], inplace=True)


In [68]:
df.head()

,Product ID,Price,Old Price,Thời gian sử dụng pin (tháng),Đối tượng sử dụng:,Đường kính mặt:,Độ dày mặt:,Nguồn năng lượng:,Chất liệu mặt kính:,Thương hiệu của:,Hãng:,Quantity Sale,Discount,Rating,Review Count,Bình luận,Tên sản phẩm,Mức kháng nước (ATM)
0,242206,990000.0,1990000.0,24.0,Nam,40 mm,9 mm,Pin,Kính Sapphire,Việt Nam,MVW.,184,0.50,4.6,20 đánh giá,"Đồng hồ đúng giao đúng mẫu, đúng màu ,đẹp ,từ ...",MVW MS06001,5.0
1,269354,690000.0,1090000.0,24.0,Nữ,33 mm,7.7 mm,Pin,Kính khoáng Mineral,Việt Nam,ELIO.,32,0.36,4.1,15 đánh giá,Rất ưng ý,ELIO EC00602,3.0
2,324772,290000.0,390000.0,18.0,Trẻ em,38 mm,14 mm,Pin,Nhựa Resin,Việt Nam,SMILE KID.,"1,1k",0.25,None,0,N/A,SMILE KID SL414012SK,5.0
3,208661,1702000.0,2270000.0,24.0,Nam,42 mm,Hãng không công bố,Pin,Kính khoáng Mineral,Nhật Bản,CASIO.,219,0.25,4.8,14 đánh giá,Mẫu mã đẹp nhìn chất lượng rất tốt trong tầm giá,CASIO MTP1375L7AVDF,5.0
4,228864,852000.0,1136000.0,120.0,Nam,43.5 mm,13.5 mm,Pin,Nhựa Resin,Nhật Bản,CASIO.,221,0.25,4.3,11 đánh giá,máy ok. cái dây nó có khớp đeo thật là đau tay,CASIO AE1000W1BVDF,10.0


In [69]:
import pandas as pd
import re

# Hàm xử lý cho cột 'Đường kính mặt:'
def process_diameter(diameter):
    if pd.isna(diameter) or diameter == 'N/A':
        return None  # Trả về None cho giá trị NaN hoặc 'N/A'
    if isinstance(diameter, str):  # Kiểm tra xem có phải là chuỗi không
        # Tìm số theo định dạng "xx mm"
        match = re.match(r'(\d+(\.\d+)?)\s*mm', diameter)
        if match:
            return float(match.group(1))  # Trả về giá trị đầu tiên
        # Nếu có định dạng "xx x yy mm", ta có thể lấy giá trị đầu tiên
        match_dimension = re.search(r'(\d+(\.\d+)?)\s*[x×]\s*(\d+(\.\d+)?)\s*mm', diameter)
        if match_dimension:
            return float(match_dimension.group(1))  # Trả về giá trị đầu tiên
    return None  # Trả về None nếu không tìm thấy

# Hàm xử lý cho cột 'Độ dày mặt:'
def process_thickness(thickness):
    if pd.isna(thickness) or thickness == 'N/A':
        return None  # Trả về None cho giá trị NaN hoặc 'N/A'
    if isinstance(thickness, str):  # Kiểm tra xem có phải là chuỗi không
        match = re.match(r'(\d+(\.\d+)?)\s*mm', thickness)
        if match:
            return float(match.group(1))  # Chuyển đổi thành số thực
    return None  # Trả về None cho các giá trị không hợp lệ

# Ghi đè lên cột cũ với giá trị đã xử lý
df['Đường kính mặt:'] = df['Đường kính mặt:'].apply(process_diameter)
df['Độ dày mặt:'] = df['Độ dày mặt:'].apply(process_thickness)


In [70]:
# Đổi tên các cột với đơn vị trong ngoặc
df.rename(columns={
    'Đường kính mặt:': 'Đường kính mặt (mm)',
    'Độ dày mặt:': 'Độ dày mặt (mm)'
}, inplace=True)

In [71]:
df.head()

,Product ID,Price,Old Price,Thời gian sử dụng pin (tháng),Đối tượng sử dụng:,Đường kính mặt (mm),Độ dày mặt (mm),Nguồn năng lượng:,Chất liệu mặt kính:,Thương hiệu của:,Hãng:,Quantity Sale,Discount,Rating,Review Count,Bình luận,Tên sản phẩm,Mức kháng nước (ATM)
0,242206,990000.0,1990000.0,24.0,Nam,40.0,9.0,Pin,Kính Sapphire,Việt Nam,MVW.,184,0.50,4.6,20 đánh giá,"Đồng hồ đúng giao đúng mẫu, đúng màu ,đẹp ,từ ...",MVW MS06001,5.0
1,269354,690000.0,1090000.0,24.0,Nữ,33.0,7.7,Pin,Kính khoáng Mineral,Việt Nam,ELIO.,32,0.36,4.1,15 đánh giá,Rất ưng ý,ELIO EC00602,3.0
2,324772,290000.0,390000.0,18.0,Trẻ em,38.0,14.0,Pin,Nhựa Resin,Việt Nam,SMILE KID.,"1,1k",0.25,None,0,N/A,SMILE KID SL414012SK,5.0
3,208661,1702000.0,2270000.0,24.0,Nam,42.0,NaN,Pin,Kính khoáng Mineral,Nhật Bản,CASIO.,219,0.25,4.8,14 đánh giá,Mẫu mã đẹp nhìn chất lượng rất tốt trong tầm giá,CASIO MTP1375L7AVDF,5.0
4,228864,852000.0,1136000.0,120.0,Nam,43.5,13.5,Pin,Nhựa Resin,Nhật Bản,CASIO.,221,0.25,4.3,11 đánh giá,máy ok. cái dây nó có khớp đeo thật là đau tay,CASIO AE1000W1BVDF,10.0


In [72]:
# Hàm xử lý cho cột 'Hãng' thay dấu chấm '.' bằng khoảng trắng
def process_brand(brand):
    if isinstance(brand, str):
        # Thay dấu chấm '.' bằng khoảng trắng
        return brand.replace('.', ' ').strip()  # Thêm strip() để loại bỏ các ký tự không mong muốn ở đầu và cuối
    return brand  # Nếu không phải chuỗi, trả về giá trị gốc

# Áp dụng hàm xử lý cho cột 'Hãng'
df['Hãng:'] = df['Hãng:'].apply(process_brand)


In [73]:
import re

# Hàm xử lý cho cột 'Quantity Sale'
def process_quantity_sale(quantity):
    if isinstance(quantity, str):
        # Loại bỏ ký tự trắng, và thay dấu phẩy ',' bằng dấu chấm '.' nếu có
        quantity = quantity.replace(',', '.').strip().lower()

        # Kiểm tra nếu có ký hiệu 'k' (nghìn) hoặc 'm' (triệu)
        if 'k' in quantity:
            # Lấy số trước 'k' và nhân với 1000
            match = re.match(r'(\d+(\.\d+)?)\s*k', quantity)
            if match:
                return float(match.group(1)) * 1000  # Nhân với 1000
        elif 'm' in quantity:
            # Lấy số trước 'm' và nhân với 1000000
            match = re.match(r'(\d+(\.\d+)?)\s*m', quantity)
            if match:
                return float(match.group(1)) * 1000000

        # Nếu không có ký hiệu 'k' hoặc 'm', trả về giá trị số sau khi xử lý dấu phẩy
        try:
            return float(quantity)  # Chuyển thành số sau khi thay đổi dấu phẩy thành dấu chấm
        except ValueError:
            return 0  # Nếu không thể chuyển thành số, trả về 0

    # Nếu không phải chuỗi, trả về 0
    return 0

# Áp dụng hàm xử lý cho cột 'Quantity Sale'
df['Quantity Sale'] = df['Quantity Sale'].apply(process_quantity_sale)


In [74]:
import pandas as pd

# Hàm xử lý cho cột 'Rating'
def process_rating(rating):
    # Kiểm tra nếu rating là NaN hoặc rỗng
    if pd.isna(rating) or rating == 'N/A':
        return None  # Trả về None nếu giá trị là NaN hoặc 'N/A'
    
    try:
        # Chuyển rating thành kiểu số thực (float)
        return float(rating)
    except ValueError:
        # Nếu không thể chuyển đổi, trả về None hoặc giá trị mặc định
        return None

# Áp dụng hàm xử lý cho cột 'Rating'
df['Rating'] = df['Rating'].apply(process_rating)

df['Rating'].fillna(0, inplace=True)  


In [75]:
import re

# Hàm xử lý cho cột 'Review Count'
def process_review_count(review_count):
    # Kiểm tra nếu giá trị là NaN hoặc chuỗi rỗng
    if pd.isna(review_count) or review_count == '':
        return 0  # Trả về 0 nếu không có giá trị
    
    # Dùng regex để tìm số nguyên trong chuỗi
    match = re.match(r'(\d+)', review_count)
    if match:
        return int(match.group(1))  # Trả về số nguyên
    return 0  # Nếu không tìm thấy số, trả về 0

# Áp dụng hàm xử lý cho cột 'Review Count'
df['Review Count'] = df['Review Count'].apply(process_review_count)


In [76]:
# Chuyển cột 'Product ID' về kiểu chuỗi
df['Product ID'] = df['Product ID'].astype(int)


In [77]:
# Thay thế các bình luận rỗng hoặc không ghi gì bằng "N/A"
df['Bình luận'] = df['Bình luận'].replace('', 'N/A')  # Thay thế chuỗi rỗng
df['Bình luận'] = df['Bình luận'].fillna('N/A')       # Thay thế giá trị None
df.head(5)

,Product ID,Price,Old Price,Thời gian sử dụng pin (tháng),Đối tượng sử dụng:,Đường kính mặt (mm),Độ dày mặt (mm),Nguồn năng lượng:,Chất liệu mặt kính:,Thương hiệu của:,Hãng:,Quantity Sale,Discount,Rating,Review Count,Bình luận,Tên sản phẩm,Mức kháng nước (ATM)
0,242206,990000.0,1990000.0,24.0,Nam,40.0,9.0,Pin,Kính Sapphire,Việt Nam,MVW,184.0,0.50,4.6,20,"Đồng hồ đúng giao đúng mẫu, đúng màu ,đẹp ,từ ...",MVW MS06001,5.0
1,269354,690000.0,1090000.0,24.0,Nữ,33.0,7.7,Pin,Kính khoáng Mineral,Việt Nam,ELIO,32.0,0.36,4.1,15,Rất ưng ý,ELIO EC00602,3.0
2,324772,290000.0,390000.0,18.0,Trẻ em,38.0,14.0,Pin,Nhựa Resin,Việt Nam,SMILE KID,1100.0,0.25,0.0,0,N/A,SMILE KID SL414012SK,5.0
3,208661,1702000.0,2270000.0,24.0,Nam,42.0,NaN,Pin,Kính khoáng Mineral,Nhật Bản,CASIO,219.0,0.25,4.8,14,Mẫu mã đẹp nhìn chất lượng rất tốt trong tầm giá,CASIO MTP1375L7AVDF,5.0
4,228864,852000.0,1136000.0,120.0,Nam,43.5,13.5,Pin,Nhựa Resin,Nhật Bản,CASIO,221.0,0.25,4.3,11,máy ok. cái dây nó có khớp đeo thật là đau tay,CASIO AE1000W1BVDF,10.0


In [78]:
# Thứ tự cột mong muốn
new_order = [
    'Product ID',
    'Tên sản phẩm', 
    'Price', 'Old Price','Quantity Sale', 'Discount','Thời gian sử dụng pin (tháng)', 
    'Đối tượng sử dụng:', 'Đường kính mặt (mm)', 
    'Độ dày mặt (mm)', 'Nguồn năng lượng:', 
    'Mức kháng nước (ATM)', 
    'Chất liệu mặt kính:', 'Thương hiệu của:', 'Hãng:','Review Count','Rating',
    'Bình luận'
]

# Sắp xếp lại cột
df = df[new_order]
df.head()

,Product ID,Tên sản phẩm,Price,Old Price,Quantity Sale,Discount,Thời gian sử dụng pin (tháng),Đối tượng sử dụng:,Đường kính mặt (mm),Độ dày mặt (mm),Nguồn năng lượng:,Mức kháng nước (ATM),Chất liệu mặt kính:,Thương hiệu của:,Hãng:,Review Count,Rating,Bình luận
0,242206,MVW MS06001,990000.0,1990000.0,184.0,0.50,24.0,Nam,40.0,9.0,Pin,5.0,Kính Sapphire,Việt Nam,MVW,20,4.6,"Đồng hồ đúng giao đúng mẫu, đúng màu ,đẹp ,từ ..."
1,269354,ELIO EC00602,690000.0,1090000.0,32.0,0.36,24.0,Nữ,33.0,7.7,Pin,3.0,Kính khoáng Mineral,Việt Nam,ELIO,15,4.1,Rất ưng ý
2,324772,SMILE KID SL414012SK,290000.0,390000.0,1100.0,0.25,18.0,Trẻ em,38.0,14.0,Pin,5.0,Nhựa Resin,Việt Nam,SMILE KID,0,0.0,N/A
3,208661,CASIO MTP1375L7AVDF,1702000.0,2270000.0,219.0,0.25,24.0,Nam,42.0,NaN,Pin,5.0,Kính khoáng Mineral,Nhật Bản,CASIO,14,4.8,Mẫu mã đẹp nhìn chất lượng rất tốt trong tầm giá
4,228864,CASIO AE1000W1BVDF,852000.0,1136000.0,221.0,0.25,120.0,Nam,43.5,13.5,Pin,10.0,Nhựa Resin,Nhật Bản,CASIO,11,4.3,máy ok. cái dây nó có khớp đeo thật là đau tay


In [79]:
# Xử lý tất cả các cột: thay thế các giá trị rỗng hoặc None bằng "N/A"
df.fillna('N/A', inplace=True)  # Thay thế giá trị None
df.replace('', 'N/A', inplace=True)  # Thay thế chuỗi rỗng

C:\Users\T&T\AppData\Local\Temp\ipykernel_11320\1666721214.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'N/A' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('N/A', inplace=True)  # Thay thế giá trị None


In [80]:
import psycopg2
import pandas as pd

# Kết nối tới PostgreSQL
try:
    conn = psycopg2.connect(
        dbname="DongHoDeoTay_TGDD",
        user="postgres",
        password="123",
        host="localhost",
        port="5432"
    )
    cursor = conn.cursor()
    print("Kết nối PostgreSQL thành công!")
except Exception as e:
    print("Lỗi khi kết nối PostgreSQL:", e)
    exit()

# Hàm chuyển đổi giá trị không hợp lệ
def clean_float(value):
    try:
        return float(value)
    except (ValueError, TypeError):
        return None  # Trả về None nếu giá trị không hợp lệ

try:
    # Làm sạch các cột chứa giá trị không hợp lệ
    numeric_columns = [
        "Price", "Old Price","Discount" ,"Thời gian sử dụng pin (tháng)",
        "Đường kính mặt (mm)", "Độ dày mặt (mm)", "Mức kháng nước (ATM)"
    ]
    for col in numeric_columns:
        df[col] = df[col].apply(clean_float)

    # Chèn dữ liệu vào bảng `origin`
    origin_data = df[["Thương hiệu của:"]].drop_duplicates()
    for _, row in origin_data.iterrows():
        cursor.execute("""
            INSERT INTO origin (origin_name)
            VALUES (%s)
            ON CONFLICT (origin_name) DO NOTHING
        """, (row["Thương hiệu của:"],))

    # Chèn dữ liệu vào bảng `brand`
    brand_data = df[["Hãng:"]].drop_duplicates()
    for _, row in brand_data.iterrows():
        cursor.execute("""
            INSERT INTO brand (brand_name)
            VALUES (%s)
            ON CONFLICT (brand_name) DO NOTHING
        """, (row["Hãng:"],))

    # Lấy ID từ bảng `origin` và `brand`
    cursor.execute("SELECT origin_name, originid FROM origin")
    origin_map = dict(cursor.fetchall())
    cursor.execute("SELECT brand_name, brandid FROM brand")
    brand_map = dict(cursor.fetchall())

    # Chèn dữ liệu vào bảng `product`
    for _, row in df.iterrows():
        cursor.execute("""
            INSERT INTO product (productid, name, price, old_price, brandid, originid)
            VALUES (%s, %s, %s, %s, %s, %s)
            ON CONFLICT (productid) DO NOTHING
        """, (
            row["Product ID"],
            row["Tên sản phẩm"],
            row["Price"],
            row["Old Price"],
            brand_map.get(row["Hãng:"]),  # Lấy `brandid` từ bảng `brand`
            origin_map.get(row["Thương hiệu của:"])  # Lấy `originid` từ bảng `origin`
        ))

    # Chèn dữ liệu vào bảng `product_detail`
    # Chèn dữ liệu vào bảng `product_detail`
    # Chèn dữ liệu vào bảng `product_detail`
    for _, row in df.iterrows():
        cursor.execute("""
            INSERT INTO product_detail (
                productid, quantity_sale, discount, battery_life, target_user,
                diameter, thickness, power_source, water_resistance, glass_material
            )
            VALUES (%s, %s,%s, %s, %s, %s, %s, %s, %s, %s)
            ON CONFLICT (productid) DO NOTHING
        """, (
            row["Product ID"],
            None if pd.isna(row.get("Quantity Sale", None)) else row["Quantity Sale"],
            None if pd.isna(row.get("Discount", None)) else row["Discount"],
            None if pd.isna(row.get("Thời gian sử dụng pin (tháng)", None)) else row["Thời gian sử dụng pin (tháng)"],
            None if pd.isna(row.get("Đối tượng sử dụng:", "")) else row["Đối tượng sử dụng:"],
            None if pd.isna(row.get("Đường kính mặt (mm)", "")) else row["Đường kính mặt (mm)"],
            None if pd.isna(row.get("Độ dày mặt (mm)", "")) else row["Độ dày mặt (mm)"],
            None if pd.isna(row.get("Nguồn năng lượng:", "")) else row["Nguồn năng lượng:"],
            None if pd.isna(row.get("Mức kháng nước (ATM)", "")) else row["Mức kháng nước (ATM)"],
            None if pd.isna(row.get("Chất liệu mặt kính:", "")) else row["Chất liệu mặt kính:"]
        ))



    # Chèn dữ liệu vào bảng `review`
    for _, row in df.iterrows():
        cursor.execute("""
            INSERT INTO review (productid, review_count, rating, comment)
            VALUES (%s, %s, %s, %s)
        """, (
            row["Product ID"],
            row.get("Review Count", None),
            row.get("Rating", None),
            row["Bình luận"]
        ))

    # Commit các thay đổi
    conn.commit()
    print("Dữ liệu đã được lưu vào PostgreSQL.")

except Exception as e:
    print("Lỗi khi chèn dữ liệu:", e)
    conn.rollback()

# Đóng kết nối
cursor.close()
conn.close()


Kết nối PostgreSQL thành công!
Dữ liệu đã được lưu vào PostgreSQL.


In [81]:
# Lưu DataFrame vào file Excel mới
df.to_excel('processed_data.xlsx', index=False)

print("Dữ liệu đã được lưu vào file 'processed_data.xlsx'.")

Dữ liệu đã được lưu vào file 'processed_data.xlsx'.


In [82]:
import pandas as pd
import json

# Giả sử bạn đã có DataFrame 'df' đã được xử lý

# Lưu dữ liệu vào file JSON
with open('DHDT_json_Processed.json', 'w', encoding='utf-8') as json_file:
    json.dump(df.to_dict(orient='records'), json_file, ensure_ascii=False, indent=4)

# Lưu DataFrame ra định dạng CSV
df.to_csv('DHDT_csv_Processed.csv', index=False, encoding='utf-8')

print("Dữ liệu đã được lưu vào file JSON và CSV.")


Dữ liệu đã được lưu vào file JSON và CSV.
